In [144]:
from connector.replay import Replay, State, Team
import os
import numpy as np

from typing import Optional

In [145]:
def remove_menu_and_pause(replay, debug=False):
    episodes    = []
    cur_episode = []
    prev_state  = replay[0].state
    for frame in replay:
        # Last frame was added, start new episode
        if frame.state != State.Goal and prev_state == State.Goal:
            episodes.append(cur_episode)
            cur_episode = []
        if frame.state == State.Menu and prev_state != State.Menu:
            episodes.append(cur_episode)
            cur_episode = []
            
        if debug:
            if frame.state != prev_state:
                print("Cur: {}; Prev: {}".format(frame.state, prev_state))
                print(frame.score)
            
            
        # Add only warm-up, game, and goal state
        if frame.state != State.Menu and frame.state != State.Pause:
            cur_episode.append(frame)
            
        prev_state = frame.state
        
    return episodes

def who_scored(episode) -> Optional[Team]:
    if episode[0].score == episode[-1].score:
        return None
    if episode[0].score[0] != episode[-1].score[0]:
        return Team.Red
    return Team.Blue

In [130]:
replay_parser = Replay(open("replays/cups/edition7/2018-12-02.bin", mode="rb").read())

In [131]:
replay_parser[600].__dict__

{'replayTime': 36.0,
 'state': <State.Warmup: 2>,
 'gameTime': 0.0,
 'score': (0, 0),
 'overtime': False,
 'players': [<connector.replay.Player at 0x7f4fb0e4fda0>,
 'ball': <connector.replay.Disc at 0x7f4fb0e4fe48>}

In [138]:
episodes = remove_menu_and_pause(replay_parser)

In [146]:
for episode in episodes:
    print(episode[-1].score)
    print(who_scored(episode))

(1, 0)
Team.Red
(1, 1)
Team.Blue
(1, 2)
Team.Blue
(1, 3)
Team.Blue
(1, 4)
Team.Blue
(1, 5)
Team.Blue
(1, 6)
Team.Blue
(2, 6)
Team.Red
(3, 6)
Team.Red
(3, 7)
Team.Blue
(3, 8)
Team.Blue
(4, 8)
Team.Red
(5, 8)
Team.Red
(5, 8)
None


IndexError: list index out of range

In [4]:
def transform_blue_pivotal(frames) -> Replay:
    """
    Inplace transformation of disk positions and velocities based on the pivotal team side.
    Inverse x-axis basically.
    
    frames: a list of replay frames
    """
    for frame in frames:
        for player in frame.players:
            # Inverse position and velocity along x-axis
            player.disc.x  = -player.disc.x
            player.disc.vx = -player.disc.vx
            
            # Swap left and right actions
            temp            = player.input[2]
            player.input[2] = player.input[3]
            player.input[3] = temp
            
        # Inverse position and velocity of the ball along x-axis
        frame.ball.x  = -frame.ball.x
        frame.ball.vx = -frame.ball.vx